In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install torchtext

In [ ]:
!pip install rouge

In [ ]:
import os
file=open('../input/final-data1/final_data1.pickle','rb')
import pickle
data=pickle.load(file)

In [ ]:
for row in data:
        row[0]=row[0].lstrip().rstrip().lower()
        row[1]=row[1].lstrip().rstrip().lower()

In [ ]:
import random
random.shuffle(data)

import torch.optim as optim
import torch

import re

from torchtext.data.metrics import bleu_score
from rouge import Rouge

# from transformers import MT5ForConditionalGeneration, T5Tokenizer

# model=MT5ForConditionalGeneration.from_pretrained("google/mt5-small").cuda()
# tokenizer=T5Tokenizer.from_pretrained("google/mt5-small")

from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum').cuda()
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')


data_train=data[:3000]
data_val=data[3000:3500]

rouge1=Rouge()

def rouge(sentence,ref):
    s=''
    flag=0
    for char in sentence:
        if char=='<':
            flag=1
        elif char=='>':
            flag=0
        elif flag==0:
            s+=char

    gen=re.sub('\s+',' ',s).rstrip().lstrip()
#     print(gen)
    if gen=='':
        return 0,0,0
    rg=rouge1.get_scores(gen,ref)
    r1,r2,rl=rg[0]["rouge-1"]['f'], \
    rg[0]["rouge-2"]['f'],rg[0]["rouge-l"]['f']
    
    return r1,r2,rl

def val(dev_data):

    r1_,r2_,rl_=0,0,0
    candidate_corpus,references_corpus=[],[]

    for line in dev_data:
        inp,label=[line[0]],[line[1]]
        input=tokenizer.prepare_seq2seq_batch(src_texts=inp,
                                              tgt_texts=label, padding=True, return_tensors='pt')
        
        output=model.generate(input_ids=input['input_ids'].cuda(),
                              num_beams=5, early_stopping=True, max_length=20)
        out=tokenizer.batch_decode(output)
#         print(out)
        candidate_corpus.append(tokenizer.tokenize(out[0]))
        references_corpus.append([tokenizer.tokenize(label[0])])

        r1,r2,rl=rouge(out[0],line[1])
        r1_+=r1
        r2_+=r2
        rl_+=rl

    r1_/=(len(dev_data)*0.01)
    r2_/=(len(dev_data)*0.01)
    rl_/=(len(dev_data)*0.01)
    bleu=0
    bleu=100*bleu_score(candidate_corpus, references_corpus)

    return r1_,r2_,rl_,bleu

# def generate_batch(data):
#     output=random.sample(data,4)
#     inp,label=[],[]
#     for dat in output:
#             inp.append(dat[0])
#             label.append(dat[1])
            
#     return inp,label

# optimizer=optim.AdamW(model.parameters(),lr=0.00004)

# scalar=0
# val_score=0
# for i in range(2000):
#         model.train()
#         inp,label=generate_batch(data_train)
#         input=tokenizer.prepare_seq2seq_batch(src_texts=inp,
#                                               tgt_texts=label, padding=True, return_tensors='pt',max_length=300,truncation=True)
#         outputs=model(input_ids=input['input_ids'].cuda(),labels=input['labels'].cuda())
#         loss=outputs[0]

#         scalar+=loss.item()
#         if(i+1)%50==0:
#                 print('iteration={}, training loss={}'.format(i+1,scalar/(4*50)))
#                 scalar=0
# #         if(i+1)%2000==0:
# #                  r1,r2,rl,bleu=val(data_val)
# #                  print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))
                
# #                  torch.save(model.state_dict(),'model_.pth')

#         loss.backward()
#         optimizer.step()

In [ ]:
torch.save(model.state_dict(),'model_p.pth')

In [ ]:
model.load_state_dict(torch.load('./model_p.pth'))

In [ ]:
r1,r2,rl,bleu=val(data_val)
print(r1,r2,rl,bleu)